# 📘 Mini-cuaderno: Proyecto Vision IA
---

1. INTEGRACIÓN DE YOLO

- Creamos una clase `YoloDetector` (no mostrada aquí, pero vive en `vision_yolo.py`).
- Su responsabilidad: cargar un modelo YOLO ya entrenado y ofrecer un método:
    detect(image_path) -> List[Dict]

Ese método devuelve una lista de objetos detectados con:


In [ ]:
{
  "label": "nombre del objeto",
  "confidence": 0.87,
  "bbox": [x1, y1, x2, y2]
}

Así logramos que cualquier parte del proyecto pueda usar YOLO sin reescribir lógica.

----

## 2. SERVICIO DE VISION

Archivo: `vision_service.py`

- Definimos `VisionService`, que usa `YoloDetector`.
- Establecimos una "zona restringida":
    self.restricted_area = (50, 50, 300, 300)

- Flujo:
  a) Detectar objetos con YOLO.  
  b) Revisar si alguna caja intersecta con la zona restringida.  
  c) Pintar cajas y zona sobre la imagen procesada.  
  d) Guardar imagen en `uploads/processed`.  
  e) Retornar un resumen con:
      - total de objetos
      - conteo por label
      - alertas de zona restringida
      - ruta de imagen procesada

----
## 3. ZONA RESTRINGIDA
 
- Se definió como un rectángulo (x1, y1, x2, y2).
- Implementamos `_intersects_restricted_area`:
  Calcula si la caja detectada por YOLO se cruza con la zona restringida.
- Si hay intersección, añadimos alerta: "⚠️ objeto dentro/encima de zona restringida".


----
## 4. PINTAR IMÁGENES (OpenCV)
Usamos OpenCV (`cv2`) para:
- `cv2.rectangle`: dibujar rectángulos (zona restringida en rojo, detecciones en verde).
- `cv2.putText`: escribir textos (nombre del objeto, confidence, aviso "Zona Restringida").
- Finalmente, guardamos la imagen procesada con `cv2.imwrite`.


----
## 5. MODELO DE NEUMONÍA (SimpleCNN)

Archivo: `pneumonia_model.py`

- Creamos una CNN simple con:
  - 2 capas convolucionales (conv1, conv2).
  - MaxPooling para reducir tamaño.
  - Dos capas densas (fc1 y fc2).
- Output: un valor (logit) para clasificar Normal vs Neumonía.
- La activación Sigmoid no se aplica dentro del modelo, porque se usa `BCEWithLogitsLoss`
  que ya combina Sigmoid + Binary Cross Entropy.


6. ENTRENAMIENTO DEL MODELO
----
## Archivo: `train_pneumonia.py`

- Flujo:
  1. Cargar datasets con `get_loaders()` (usa carpetas train/val/test).
  2. Crear modelo `SimpleCNN`.
  3. Definir `loss` = BCEWithLogitsLoss y optimizador Adam.
  4. Loop de epochs:
     - Entrenamiento: calculamos pérdida y actualizamos pesos.
     - Validación: calculamos pérdida y accuracy.
  5. Guardamos:
     - Pesos del modelo en `infrastructure/model/pneumonia_cnn.pth`.
     - Gráfica de pérdidas y accuracy en `infrastructure/plots/pneumonia_training.png`.

- Usamos `matplotlib` para dibujar las curvas:
  - Train Loss
  - Val Loss
  - Val Accuracy


----
## 7. PREPROCESAMIENTO DE IMÁGENES

Archivo: `preprocess.py`

- Usamos OpenCV para:
  a) Leer imagen en escala de grises (`cv2.imread(..., IMREAD_GRAYSCALE)`).
  b) Redimensionar a 224x224.
  c) Normalizar valores a rango [-1, 1].
  d) Convertir a tensor de PyTorch.
  e) Añadir dimensiones para lotes (batch).


----
## 8. ANOTACIÓN EN RADIOGRAFÍAS

Archivo: `draw_xray_annotation.py`

- Función que recibe:
  - Ruta de imagen
  - Si es tórax o no
  - Predicción (Normal / Neumonía)
  - Confianza (probabilidad)
- Dibuja un texto verde si es Normal, rojo si es Neumonía o "NO ES TORAX".
- Usa `cv2.putText` y devuelve la imagen anotada.

----
## RESUMEN GENERAL
---------------
- YOLO: detección de objetos en imágenes.
- VisionService: lógica de zona restringida y resúmenes.
- OpenCV: dibujar cajas, textos y guardar imágenes procesadas.
- SimpleCNN: red neuronal simple para neumonía.
- Entrenamiento: guardamos pesos y gráficas.
- Preprocesamiento: convertir imágenes crudas a tensores listos para la CNN.
- Anotación: pintar predicciones en imágenes.